## **<h3 align="center"> Deep Learning - Project </h3>**
# **<h3 align="center"> Phylum Arthropoda - Steven</h3>**
**Group 4 members:**<br>
Alexandra Pinto - 20211599@novaims.unl.pt - 20211599<br>
Steven Carlson - 20240554@novaims.unl.pt - 20240554<br>
Sven Goerdes - 20240503@novaims.unl.pt - 20240503<br>
Tim Straub - 20240505@novaims.unl.pt - 20240505<br>
Zofia Wojcik  - 20240654@novaims.unl.pt - 20240654<br>

# Table of Contents
* [1. Introduction](#intro)
* [2. Setup](#setup)
* [3. Data Loading](#dataloading)
* [4. Image Preprocessing](#imagepreprocessing)
* [5. Neural Networks Models](#nnmodels)



# 1. Introduction <a class="anchor" id="intro"></a>

In this second notebook, we will preprocess images from the **Arthropoda** phylum and develop a deep learning model to accurately classify them at the family level.

# 2. Setup <a class="anchor" id="setup"></a>
In this section, we will import the necessary libraries that will be used throughout the notebook. These libraries will help with data handling and image processing.

In [1]:
# Standard libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns
import itertools
import random

# Libraries for image processing
from glob import glob
from PIL import Image


In [2]:
#Libraries from Keras / TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


#import tensorflow_hub as hub

#Import pre-trained models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess




2025-04-15 11:49:16.618367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 11:49:16.933699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744714157.044283     609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744714157.077361     609 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744714157.328161     609 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
tf.config.run_functions_eagerly(False)


# 3. Data Loading <a class="anchor" id="dataloading"></a>

Let's open the train and test for Arthropoda Phylum.

In [4]:
# Load the DataFrame from the CSV file
arthropoda_train = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_train.csv")
arthropoda_train.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28260809,1065329,animalia,arthropoda,apidae,arthropoda_apidae/28260809_1065329_eol-full-si...
1,29945328,1077217,animalia,arthropoda,pseudophasmatidae,arthropoda_pseudophasmatidae/29945328_1077217_...
2,14644212,463474,animalia,arthropoda,formicidae,arthropoda_formicidae/14644212_463474_eol-full...


In [5]:
# Load the DataFrame from the CSV file
arthropoda_test = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_test.csv")
arthropoda_test.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28408206,1065346,animalia,arthropoda,apidae,arthropoda_apidae/28408206_1065346_eol-full-si...
1,28253620,1065348,animalia,arthropoda,apidae,arthropoda_apidae/28253620_1065348_eol-full-si...
2,21847584,1065348,animalia,arthropoda,apidae,arthropoda_apidae/21847584_1065348_eol-full-si...


# 4. Image Preprocessing <a class="anchor" id="imagepreprocessing"></a>

In [6]:
# Map model names to their preprocessors and classes
model_map = {
    'resnet50':      (ResNet50,      resnet_preprocess),
    'MobileNetV2':   (MobileNetV2,   mobilenet_preprocess),
    'efficientnetb0':(EfficientNetB0, efficientnet_preprocess),
    'densenet121':   (DenseNet121,   densenet_preprocess),
    'inceptionv3':   (InceptionV3,   inception_preprocess),
}


In [7]:
#Define preprocess and augmentation functions

#Function to preprocess the images
def process_image(file_path, label, preprocess_fn):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)

    if preprocess_fn:
        image = preprocess_fn(image)
    else:
        # default normalization
        image = tf.cast(image, tf.float32) / 255.0

    return image, label




#Function to augment the images
def augment_image(image, label):

    #Randomly change brightness
    image = tf.image.random_brightness(image, max_delta=0.2)

    #Apply geometric augmentations
    image = geometric_augmentation_layers(image, training=True) # Apply geometric augmentations
    image = tf.clip_by_value(image, 0.0, 1.0)
    
    return image, label


# Geometric augmentations
geometric_augmentation_layers = tf.keras.Sequential(
    [
        # Randomly flip horizontally
        tf.keras.layers.RandomFlip("horizontal"),

        # Randomly rotate
        tf.keras.layers.RandomRotation(factor=0.12),

        # Random zoom
        tf.keras.layers.RandomZoom(height_factor=(-0.35, 0.35), # Corresponds to [0.8, 1.2] of original height
                                   width_factor=(-0.35, 0.35)), # Corresponds to [0.8, 1.2] of original width

        # Random shift
        tf.keras.layers.RandomTranslation(height_factor=0.20,
                                          width_factor=0.20),

        # Contrast
        tf.keras.layers.RandomContrast(factor=0.25),

    ],
    name="geometric_augmentations",
)


I0000 00:00:1744714162.820620     609 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1744714162.826579     609 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [8]:
#Define some stuff
num_classes = arthropoda_train['family'].nunique() #number of classes = number of families
batch_size = 64
input_shape = (224, 224, 3)
image_size = (224, 224)
value_range = (0.0, 1.0)
num_classes = 17  

# Define callbacks
my_callbacks = [
callbacks.EarlyStopping(patience=10, restore_best_weights=True),
callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
]



# Import data

In [9]:
root_dir = "/home/sacar/DeepLearning2425/rare_species"

#Get the file paths and labels for the training and test set
arthropoda_train['full_path'] = arthropoda_train['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))
arthropoda_test['full_path'] = arthropoda_test['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))

file_paths_train = arthropoda_train['full_path'].tolist()
labels_train = arthropoda_train['family'].tolist()

file_paths_test = arthropoda_test['full_path'].tolist()
labels_test = arthropoda_test['family'].tolist()

#Map the labels to integers
label_names = sorted(set(labels_train)) # Get the unique labels
label_to_index = {name: i for i, name in enumerate(label_names)} # Create a mapping from labels to integers
labels_train = [label_to_index[label] for label in labels_train]
labels_test = [label_to_index[label] for label in labels_test]

#Convert the labels to numpy arrays
labels_train = np.array(list(labels_train), dtype=np.int32)
labels_test = np.array(list(labels_test), dtype=np.int32)


print(file_paths_train[:5])
print(labels_train[:5])
print(file_paths_test[:5])
print(labels_test[:5])

['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28260809_1065329_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945328_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14644212_463474_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945335_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_papilionidae/21035374_130548_eol-full-size-copy.jpg']
[ 0 13  5 13 10]
['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28408206_1065346_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28253620_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/21847584_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14681521_403723_eol-full-size-copy.jpg', '/home/sacar/DeepLear

# Function to build dataset

In [10]:
#Function to build the dataset
def build_dataset(file_paths, labels, preprocess_fn, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: process_image(x, y, preprocess_fn), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(buffer_size=1000, reshuffle_each_iteration=True, seed=42)
    dataset = dataset.batch(16).prefetch(tf.data.AUTOTUNE)
    return dataset



# Confirm GPU is working

In [11]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [12]:
# Print GPU devices detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU(s) detected: {[gpu.name for gpu in gpus]}")
else:
    print("❌ No GPU detected by TensorFlow.")

✅ GPU(s) detected: ['/physical_device:GPU:0']


# Run Models and Plot Results

In [13]:
#Create the random search function
def run_random_search(train, val, num_classes, base_model_class, n_trials=15, seed=42):
    random.seed(seed)

    # Hyperparameter search space
    dropout_rates = [0.5, 0.6, 0.7]
    dense_units_list = [64, 128]
    learning_rates = [1e-5, 5e-5, 1e-4]
    patience_values = [5, 7]
    freeze_until_layers = [100, 120, 140]
    optimizers_list = ['adam']

    # Generate all possible combinations
    all_combinations = list(itertools.product(
        dropout_rates,
        dense_units_list,
        learning_rates,
        patience_values,
        freeze_until_layers,
        optimizers_list
    ))

    # Randomly sample N trials
    sampled_combinations = random.sample(all_combinations, k=min(n_trials, len(all_combinations)))
    results = []

    for i, (dropout, units, lr, patience, freeze_until, opt_name) in enumerate(sampled_combinations):
        print(f"\n Trial {i+1}/{len(sampled_combinations)}")
        print(f"Dropout={dropout}, Units={units}, LR={lr}, Patience={patience}, FreezeUntil={freeze_until}")

        # Rebuild base model for fresh training
        base_model = base_model_class(
            input_shape=(224, 224, 3),
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False

        def create_optimizer():
            return keras.optimizers.Adam(learning_rate=lr)

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(dropout),
            layers.Dense(units, activation='relu'),
            layers.Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer = create_optimizer(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        callbacks_list = [
            EarlyStopping(patience=patience, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=max(1, patience // 2)),
            ModelCheckpoint("temp_best_model.keras", save_best_only=True)
        ]


        # Phase 1 training
        history1 = model.fit(
            train,
            validation_data=val,
            epochs=30,
            callbacks=callbacks_list,
            verbose=0
        )

        # Phase 2 fine-tuning
        base_model.trainable = True
        for layer in base_model.layers[:freeze_until]:
            layer.trainable = False

        model.compile(
            optimizer = create_optimizer(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        history2 = model.fit(
            train,
            validation_data=val,
            epochs=30,
            callbacks=callbacks_list,
            verbose=0
        )

        final_val_acc = history2.history['val_accuracy'][-1]

        results.append({
            'dropout': dropout,
            'dense_units': units,
            'learning_rate': lr,
            'patience': patience,
            'freeze_until': freeze_until,
            'optimizer': opt_name,
            'val_accuracy': final_val_acc
        })

    return pd.DataFrame(results)


In [14]:
def plot_random_search_results(results_df, top_n=5):
    sns.set(style="whitegrid")

    # -------------------------------
    # 1. Top N Configurations by Val Accuracy
    # -------------------------------
    top_configs = results_df.sort_values(by='val_accuracy', ascending=False).head(top_n)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=top_configs, x='val_accuracy', y=top_configs.index, hue='dropout')
    plt.title(f"Top {top_n} Hyperparameter Configs by Validation Accuracy")
    plt.xlabel("Validation Accuracy")
    plt.ylabel("Config Index")
    plt.legend(title="Dropout")
    plt.tight_layout()
    plt.show()

    # -------------------------------
    # 2. Strip plots: Accuracy vs Each Hyperparam
    # -------------------------------
    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle("Validation Accuracy vs Hyperparameters", fontsize=16)

    sns.stripplot(data=results_df, x='dropout', y='val_accuracy', ax=axs[0, 0])
    axs[0, 0].set_title("Dropout")

    sns.stripplot(data=results_df, x='dense_units', y='val_accuracy', ax=axs[0, 1])
    axs[0, 1].set_title("Dense Units")

    sns.stripplot(data=results_df, x='learning_rate', y='val_accuracy', ax=axs[0, 2])
    axs[0, 2].set_title("Learning Rate")

    sns.stripplot(data=results_df, x='patience', y='val_accuracy', ax=axs[1, 0])
    axs[1, 0].set_title("EarlyStopping Patience")

    sns.stripplot(data=results_df, x='freeze_until', y='val_accuracy', ax=axs[1, 1])
    axs[1, 1].set_title("Freeze Until Layer")

    axs[1, 2].axis('off')  # Empty slot

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # -------------------------------
    # 3. Heatmap (e.g. Dropout vs Units)
    # -------------------------------
    pivot_table = results_df.pivot_table(
        values='val_accuracy',
        index='dropout',
        columns='dense_units',
        aggfunc='mean'
    )

    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="viridis")
    plt.title("Heatmap: Dropout vs Dense Units (Val Accuracy)")
    plt.xlabel("Dense Units")
    plt.ylabel("Dropout Rate")
    plt.tight_layout()
    plt.show()


In [ ]:
models_list = ['inceptionv3', 'MobileNetV2', 'efficientnetb0', 'densenet121', 'resnet50']

for model_name in models_list:
    print(f"\n>>> Running model: {model_name}\n")

    # Get model constructor and preprocessing function
    base_model_class, preprocess_fn = model_map[model_name]

    # Build datasets
    train = build_dataset(file_paths_train, labels_train, preprocess_fn, augment=True)
    val   = build_dataset(file_paths_train, labels_train, preprocess_fn, augment=False)
    test  = build_dataset(file_paths_test,  labels_test,  preprocess_fn, augment=False)

    # Run hyperparameter search
    results = run_random_search(
        train, val, num_classes,
        base_model_class=base_model_class,
        n_trials=15,
        seed=42
    )

    # Plot results
    plot_random_search_results(results, top_n=1)





>>> Running model: inceptionv3


 Trial 1/15
Dropout=0.7, Units=64, LR=5e-05, Patience=7, FreezeUntil=100


I0000 00:00:1744714182.854640     833 service.cc:152] XLA service 0x7fb94c002c00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744714182.855333     833 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-04-15 11:49:43.254123: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744714186.092527     833 cuda_dnn.cc:529] Loaded cuDNN version 90300
